# HW2 Association Rules

1. 使⽤weka回答以下問題：
    - (a)請嘗試著修改 adult.csv 的欄位與上圖相同，並轉換成 arff 檔使 weka可以執⾏ Association Rule，請說明使⽤⽅法以及解釋原來的檔案不能執⾏的原因︖(10%)
    - (b) 請將 numRules 設成 5 和 10，其各別執⾏後的 Minimum support 為何，請比較兩者並說明造成其差異的原因。(15%)
    - (c) 將 numRule 設成 10，列出前 5 條 rule(15%)
    - (d) 如何在 Associator output 產⽣ Itemset，請截圖說明並附上 Itemset 結果。(15%)


2. 使⽤python回答以下問題：
    - (e) 使⽤已修改過的 adult.csv 檔，使⽤ Apriori 演算法進⾏分析,設定 confidence = 0.9、minimum support = 0.2，過程中對所有重要程式步驟進⾏截圖並加以說明，越詳盡越好。(15%)
    - (f) 產⽣與 (c) ⼩題⼀樣的結果，列出前五條best rules，截圖並加以說明(15%)

## 1. Weka
### (a) 請嘗試著修改 adult.csv 的欄位與上圖相同，並轉換成 arff 檔使 weka可以執行 Association Rule，請說明使用方法以及解釋原來的檔案不能執行的原因? (10%)

一開始我們只有adult.csv檔案，匯入時發現Association Rules的start鍵按不下去，因為csv裡面gender的欄位資料只有0和1；0代表Male，1代表Female，但是weka判斷此欄位為numeric型態，所以執行不了。
//img

因此需要先將numeric型態，轉成nominal型態。Weka有內鍵功能來轉換attribute的型態，我們先選取gender欄位，點選filter > nonsupervise > numeric to nominal按下執行即可。如此一來就可將0和1轉成nominal的形式。
```python
@attribute gender {0, 1}
```
//img

接下來，由於gender的值還是0和1，雖然可以run但可讀性還是差了一些，所以接下來我們再把0和1改名成Male和Female，選取filter > nonsupervise > rename nominal values。按照下圖輸入參數。最後再save成.arff檔即可。
```python
@attribute gender {Male, Female}
```
//img

另外我也手刻一隻python程式`Csv to Arff.ipynb`可以將.csv轉換成.arff，因為考量通用性，所有的attribute都是nominal型態。沒有提供改名功能。

### (b) 請將 numRules 設成 5 和 10，其各別執⾏後的 Minimum support 為何，請比較兩者並說明造成其差異的原因。(15%)

### (c) 將 numRule 設成 10，列出前 5 條 rule(15%)

```python
1. marital-status=Never-married hours-per-week=20-40 9669 ==> income=<=50K 9368    <conf:(0.97)> lift:(1.29) lev:(0.05) [2098] conv:(7.94)
                
2. workclass=Private marital-status=Never-married 12243 ==> income=<=50K 11755    <conf:(0.96)> lift:(1.28) lev:(0.06) [2549] conv:(6.21)
                
3. workclass=Private marital-status=Never-married race=White 10134 ==> income=<=50K 9702    <conf:(0.96)> lift:(1.27) lev:(0.05) [2082] conv:(5.81)
                
4. marital-status=Never-married 14875 ==> income=<=50K 14153    <conf:(0.95)> lift:(1.27) lev:(0.06) [2968] conv:(5.1)
                
5. marital-status=Never-married race=White 12228 ==> income=<=50K 11590    <conf:(0.95)> lift:(1.26) lev:(0.05) [2396] conv:(4.75)
                ```